# Deploying a DL model to AKS CPU cluster
This notebook shows the steps for deploying a service: registering a model, creating an image, provisioning a cluster (one time action), and deploying a service to it. We then test and delete the service, image and model.

In [ ]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model

In [ ]:
import azureml.core
print(azureml.core.VERSION)

In [ ]:
import os
print(os.getcwd())
print(os.listdir(os.getcwd()))

## Get workspace
Load existing workspace from the config file info.

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create the model

In [ ]:
#Creating the model pickle file
import tensorflow as tf
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

model = ResNet152(weights='imagenet')

In [ ]:
model.save_weights("model_resnet_weights.h5")

In [ ]:
model.load_weights('model_resnet_weights.h5')

In [ ]:
model

## Register the model
Register an existing trained model, add descirption and tags.

In [ ]:
#Register the model
from azureml.core.model import Model
model = Model.register(model_path = "model_resnet_weights.h5", # this points to a local file
                       model_name = "resnet_model", # this is the name the model is registered as
                       tags = {'model': "dl", 'framework': "resnet"},
                       description = "resnet 152 model",
                       workspace = ws)

print(model.name, model.description, model.version)

## Test the scoring script at local host

In [ ]:
#define init() function
def init():
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging

    global model
    model = ResNet152(weights='imagenet')
    print('Model loaded')

In [ ]:
init()

In [ ]:
#define run() function 
def run(inputString):
    
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging   
    
    model = ResNet152(weights='imagenet')
    print('Model loaded')
  
    
     
    responses = []
    base64Dict = json.loads(inputString)

    for k, v in base64Dict.items():
        img_file_name, base64Img = k, v
    decoded_img = base64.b64decode(base64Img)
    img_buffer = BytesIO(decoded_img)
    imageData = Image.open(img_buffer).convert("RGB")
    
    # Evaluate the model using the input data
    img = ImageOps.fit(imageData, (224,224), Image.ANTIALIAS)
    img = np.array(img) # shape: (224, 224, 3)
    
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    preds = model.predict(img)
    print('Predicted:', decode_predictions(preds, top=3))
    resp = {img_file_name: str(decode_predictions(preds, top=3))}

    responses.append(resp)
    return json.dumps(responses)

In [ ]:
from io import BytesIO
from PIL import Image, ImageOps
import base64
import json

img_path = '220px-Lynx_lynx_poing.jpg'
encoded = None
with open(img_path, 'rb') as file:
  encoded = base64.b64encode(file.read())
img_dict = {img_path: encoded.decode('utf-8')}
body = json.dumps(img_dict)
resp = run(body)
print(resp)

## Write and save scoring script

In [ ]:
%%writefile score.py
def init():
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging

    global model
    model = ResNet152(weights='imagenet')
    print('Model loaded')
    
def run(inputString):
    
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging   
    
    model = ResNet152(weights='imagenet')
    print('Model loaded')
  
    responses = []
    base64Dict = json.loads(inputString)

    for k, v in base64Dict.items():
        img_file_name, base64Img = k, v
    decoded_img = base64.b64decode(base64Img)
    img_buffer = BytesIO(decoded_img)
    imageData = Image.open(img_buffer).convert("RGB")
    
    # Evaluate the model using the input data
    img = ImageOps.fit(imageData, (224,224), Image.ANTIALIAS)
    img = np.array(img) # shape: (224, 224, 3)
    
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    preds = model.predict(img)
    print('Predicted:', decode_predictions(preds, top=3))
    resp = {img_file_name: str(decode_predictions(preds, top=3))}

    responses.append(resp)
    return json.dumps(responses)    

## Create an Image
Create an image using the registered model the script that will load and run the model.

In [ ]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv_yz.yml",
                                                  docker_file = "mydockerfile",
                                                  description = "Image for AKS Deployment Tutorial",
                                                  tags = {"name":"AKS","project":"AML"}, 
                                                  dependencies = ["resnet152.py"]                                                  
                                                 )

image = ContainerImage.create(name = "myimage12",
                              # this is the model object
                              models = [],                              
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

In [ ]:
image.wait_for_creation(show_output = True)

## Provision the AKS Cluster¶ 
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

In [ ]:
#Provision AKS cluster
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'yanz-aks-cpu' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [ ]:
'''
#Provision AKS cluster with GPU machine
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration(vm_size='Standard_NC6')

aks_name = 'yanz-aks-1' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)
'''

## Optional step: Attach existing AKS cluster¶

In [ ]:
# Attach an existing AKS cluster
# Use the default configuration (can also provide parameters to customize)
resource_id = '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/yanzamlworkspace/providers/Microsoft.ContainerService/managedClusters/yanz-aks-1c6750233554'

create_name='my-existing-aks' 
# Create the cluster
aks_target = AksCompute.attach(workspace=ws, name=create_name, resource_id=resource_id)
# Wait for the operation to complete
aks_target.wait_for_completion(True)


In [ ]:

# Attach an existing AKS cluster (CPU)
# Use the default configuration (can also provide parameters to customize)
resource_id = '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/yanzamlworkspace/providers/Microsoft.ContainerService/managedClusters/yanz-aks-cpu1549812594'

create_name='my-cpu-aks' 
# Create the cluster
aks_target = AksCompute.attach(workspace=ws, name=create_name, resource_id=resource_id)
# Wait for the operation to complete
aks_target.wait_for_completion(True)


In [ ]:
aks_target

In [ ]:
#list images
images = ws.images()
images

In [ ]:
#for img in ws.images():
#    if img.name == 'myimage1': img.delete()

In [ ]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

In [ ]:
!az aks get-credentials -n 'yanz-aks-cpu1549812594' -g yanzamlworkspace -a -f config_cpuaks

In [ ]:
!kubectl --kubeconfig config get services

## Deploy web service to AKS¶ 

In [ ]:
'''
#Deploy web service to AKS
#Set the web service configuration (using default here)
aks_config = AksWebservice.deploy_configuration()
print(aks_config)
'''

In [ ]:
#Deploy web service to AKS
#Set the web service configuration (using customized configuration)
aks_config = AksWebservice.deploy_configuration(memory_gb=2.0, enable_app_insights=True)
print(aks_config)

In [ ]:
# by default the 500MB -- memory_gb; up to 1.4 GB  - manually - profiling ;  cluster capacity is 24G
help(AksWebservice)

In [ ]:
image

In [ ]:
%%time
aks_service_name ='yanz-aks-service-14'

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

## Test the web service¶ 
We test the web sevice by passing data.

In [ ]:
from io import BytesIO
from PIL import Image, ImageOps
import base64
import json

img_path = '220px-Lynx_lynx_poing.jpg'
encoded = None
with open(img_path, 'rb') as file:
  encoded = base64.b64encode(file.read())
img_dict = {img_path: encoded.decode('utf-8')}
body = json.dumps(img_dict)
resp = aks_service.run(input_data = body)
print(resp)

In [ ]:
aks_service.update(enable_app_insights=True)

In [ ]:
print(aks_service.state)

In [ ]:
aks_service.update_deployment_state()

In [ ]:
!kubectl --kubeconfig config_cpuaks proxy --port 8011

In [ ]:
log = aks_service.get_logs(5000)

In [ ]:
with open("servicelog_ws14", "w") as json_file:
    json_file.write(log)

In [ ]:
#debug web service failure
print(ws.webservices()['yanz-aks-service-11'].get_logs())

In [ ]:
#clean up resources
#aks_target = AksCompute(name='jaya-aks-1',workspace=ws)
#aks_target.delete()

In [ ]:
#alternate code to clean up resources
#!az aks delete --resource-group jayavienna --name jaya-aks-2 --yes

In [ ]:
#for s in ws.webservices():
#    print(s.name)

In [ ]:
#s =  Webservice(ws, 'jaya-aks-service-2')
#s.delete()

In [ ]:
#from azureml.core import Workspace
#from azureml.core.compute import AksCompute, ComputeTarget

#ws = Workspace.from_config()

#for c in ws.compute_targets():
#    print(c.name)
